In [11]:
# Importing the model
from model.ctabgan import CTABGAN
# Importing the evaluation metrics 
from model.eval.evalFidel import evaluate_fidelity
# Importing standard libraries
import numpy as np
import pandas as pd
import glob

In [10]:
pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Specifying the replication number 
num_exp = 1 
# Specifying the name of the dataset used 
dataset = "car" 
# Specifying the path of the dataset used 
real_path = "Real_Datasets/Datasets/Car/car.csv" 
# Specifying the root directory for storing generated data
fake_file_root = "Fake_Datasets" 

In [13]:
# Initializing the synthesizer object and specifying input parameters
# Notice: If you have continuous variable, you do not need to explicitly assign it. It will be treated like 
# that by default
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20,  
                 categorical_columns = ['Buying','Maint','Doors','Persons','Lug_boot','Safety','Class'], 
                 log_columns = [],
                 mixed_columns= {}, 
                 integer_columns = [],
                 problem_type= {"Classification": 'Class'},
                 epochs = 1) 

# Fitting the synthesizer to the training dataset and generating synthetic data
for i in range(num_exp):
    synthesizer.fit()
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}.csv".format(exp=i), index= False)

100%|██████████| 1/1 [00:03<00:00,  3.60s/it]

Finished training in 4.354864835739136 seconds.


In [14]:
# Collecting the paths to all corresponding generated datasets for evaluation 
fake_paths = glob.glob(fake_file_root+"/"+dataset+"/"+"*")

In [15]:
fake_paths="Fake_Datasets/"+dataset+"/"+dataset+"_fake_0.csv"

## Fidelity tests

##### Metric	Optimal Value	Interpretation
##### KS Statistic	0	Distributions are identical
##### KS p-value	~1 (≥ 0.05)	Cannot reject null; distributions match
##### Wasserstein Distance	0	No difference between distributions
##### JS Divergence	0	Distributions are identical


In [16]:
real_data=pd.read_csv(real_path)
synthetic_data = pd.read_csv(fake_paths)
fidelity_results = evaluate_fidelity(real_data, synthetic_data)
for col, metrics in fidelity_results.items():
        print(f"\nColumn: {col}")
        for metric, value in metrics.items():
            print(f"  {metric}: {value}")


Column: Buying
  chi2_stat: 1.8353797269784111
  chi2_p: 0.6072663405555392
  degrees_of_freedom: 3
  contingency_table: [[432, 432, 432, 432], [433, 463, 424, 408]]

Column: Maint
  chi2_stat: 1.107748174760003
  chi2_p: 0.7752039803553243
  degrees_of_freedom: 3
  contingency_table: [[432, 432, 432, 432], [441, 450, 409, 428]]

Column: Doors
  chi2_stat: 1.0808490165045364
  chi2_p: 0.7816991023559808
  degrees_of_freedom: 3
  contingency_table: [[432, 432, 432, 432], [424, 416, 457, 431]]

Column: Persons
  chi2_stat: 1.157972863073742
  chi2_p: 0.5604661495871848
  degrees_of_freedom: 2
  contingency_table: [[576, 576, 576], [568, 555, 605]]

Column: Lug_boot
  chi2_stat: 0.7320846554194664
  chi2_p: 0.6934734474457303
  degrees_of_freedom: 2
  contingency_table: [[576, 576, 576], [596, 555, 577]]

Column: Safety
  chi2_stat: 0.34465608564257
  chi2_p: 0.8417030132436873
  degrees_of_freedom: 2
  contingency_table: [[576, 576, 576], [591, 574, 563]]

Column: Class
  chi2_stat: 862

### Neural Network Classifier


In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from model.eval.evaluation import MixedTabularNN

# Define your column names
categorical_cols = ['Buying','Maint','Doors','Persons','Lug_boot','Safety']
numerical_cols = []  # Add any numerical columns if available
target_col = 'Class'

# ---------------------------
# 1. Load and Process Synthetic Training Data
# ---------------------------
synthetic_df = pd.read_csv(fake_paths)  # synthetic training data file

# Fit label encoders on synthetic training data (including target)
cat_encoders = {col: LabelEncoder().fit(synthetic_df[col]) for col in categorical_cols + [target_col]}

# Apply encoding on synthetic training data
for col, le in cat_encoders.items():
    synthetic_df[col] = le.transform(synthetic_df[col])

# Fit scaler on numerical features (if any)
scaler = StandardScaler()
if numerical_cols:
    synthetic_df[numerical_cols] = scaler.fit_transform(synthetic_df[numerical_cols])

# Separate features and target for synthetic training
X_cat_train = synthetic_df[categorical_cols].values  # categorical input
X_num_train = synthetic_df[numerical_cols].values if numerical_cols else np.zeros((len(synthetic_df), 0))
y_train = synthetic_df[target_col].values

# Convert synthetic training data to torch tensors
X_cat_train_tensor = torch.tensor(X_cat_train, dtype=torch.long)
X_num_train_tensor = torch.tensor(X_num_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# ---------------------------
# 2. Load and Process Real Testing Data
# ---------------------------
real_df = pd.read_csv(real_path)  # real testing data file

# IMPORTANT: Use the same encoders from synthetic training for real test data
for col, le in cat_encoders.items():
    if col in real_df.columns:
        real_df[col] = le.transform(real_df[col])

# Apply the same scaler on numerical features (if any)
if numerical_cols:
    real_df[numerical_cols] = scaler.transform(real_df[numerical_cols])

# Separate features and target for real testing (if the target is available)
X_cat_test = real_df[categorical_cols].values  # categorical input
X_num_test = real_df[numerical_cols].values if numerical_cols else np.zeros((len(real_df), 0))
y_test = real_df[target_col].values  # if available for evaluation

# Convert real testing data to torch tensors
X_cat_test_tensor = torch.tensor(X_cat_test, dtype=torch.long)
X_num_test_tensor = torch.tensor(X_num_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# ---------------------------
# 3. Create the Model and Evaluate
# ---------------------------
# Get the cardinalities for the categorical features
categorical_cardinalities = [len(cat_encoders[col].classes_) for col in categorical_cols]

# Create your model; adjust output_dim if necessary
model = MixedTabularNN(
    categorical_cardinalities=categorical_cardinalities,
    num_numerical_features=X_num_train.shape[1],
    output_dim=len(np.unique(y_train))  # for classification
)

# Now you can train your model using the synthetic training tensors and evaluate it on the real testing tensors.
# For example:
#   predictions = model(X_cat_test_tensor, X_num_test_tensor)
#   Evaluate predictions against y_test_tensor


In [18]:
real_path

'Real_Datasets/Datasets/Car/car.csv'

In [19]:
import torch.optim as optim
import torch.nn as nn

# categorical_cols is your list of categorical column names
# cat_encoders is a dict of LabelEncoders for each column

categorical_cardinalities = [len(cat_encoders[col].classes_) for col in categorical_cols]

model = MixedTabularNN(categorical_cardinalities, num_numerical_features=X_num.shape[1], output_dim=4)

# Use CrossEntropyLoss for multi-class classification
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
X_cat_train_tensor = torch.tensor(X_cat_train, dtype=torch.long)
X_num_train_tensor = torch.tensor(X_num_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_cat_test_tensor = torch.tensor(X_cat_test, dtype=torch.long)
X_num_test_tensor = torch.tensor(X_num_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)



NameError: name 'X_num' is not defined

In [32]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_num_train_tensor, X_cat_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [5/100], Loss: 1.3219
Epoch [10/100], Loss: 1.2092
Epoch [15/100], Loss: 1.1593
Epoch [20/100], Loss: 1.1577
Epoch [25/100], Loss: 1.1444
Epoch [30/100], Loss: 1.1380
Epoch [35/100], Loss: 1.1350
Epoch [40/100], Loss: 1.1290
Epoch [45/100], Loss: 1.1280
Epoch [50/100], Loss: 1.1212
Epoch [55/100], Loss: 1.1207
Epoch [60/100], Loss: 1.1209
Epoch [65/100], Loss: 1.1150
Epoch [70/100], Loss: 1.1090
Epoch [75/100], Loss: 1.1091
Epoch [80/100], Loss: 1.1095
Epoch [85/100], Loss: 1.1079
Epoch [90/100], Loss: 1.1021
Epoch [95/100], Loss: 1.1008
Epoch [100/100], Loss: 1.0974


In [33]:
from sklearn.metrics import classification_report, accuracy_score

model.eval()
with torch.no_grad():
    test_outputs = model(X_num_test_tensor, X_cat_test_tensor)
    _, predicted = torch.max(test_outputs, 1)  # Get class indices

    acc = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    print(f"Test Accuracy: {acc:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test_tensor.numpy(), predicted.numpy()))


Test Accuracy: 0.6991

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       384
           1       0.00      0.00      0.00        69
           2       0.70      1.00      0.82      1210
           3       0.00      0.00      0.00        65

    accuracy                           0.70      1728
   macro avg       0.17      0.25      0.21      1728
weighted avg       0.49      0.70      0.58      1728

